In [6]:
from proj import *
import numpy as np

In [3]:
xs = getXS()

In [4]:
xs

[<proj.CrossSection instance at 0x104cf0560>,
 <proj.CrossSection instance at 0x104cf05a8>,
 <proj.CrossSection instance at 0x104cf0638>]

In [5]:
xs[0].sigTr[0]

0.223775

In [11]:
solnMesh = Mesh(10, 
                np.linspace(0.0,400,11), 
                [2,1,1,1,3,3,1,1,1,2],
                [2,1])

In [15]:
for i in range(solnMesh.nX) :
    print xs[solnMesh.mat[i]-1].sigA[1]

0.0203
0.158201
0.158201
0.158201
0.182224
0.182224
0.158201
0.158201
0.158201
0.0203
